In [14]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from melo.api import TTS

# 초기화
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

# 톤 컬러 임베딩
reference_speaker = 'sunhi_sample.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)


# 생성
texts = { 'KR': "지금 이 음성은 저번에 사용한 티티에스 싸이트를 통해서 생성한 음성이 아니고, 샘플 음성을 제로샷으로 습득하여 로컬 티티에스 모델에서 만들어낸 목소리 입니다. 목소리를 잘 따라하는 것을 볼 수 있습니다."}

src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.1

for language, text in texts.items():
    print(language)
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)

cuda:0


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
OpenVoice version: v2


Estimating duration from bitrate, this may be inaccurate
/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/functional.py:666: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789115370/work/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


KR


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_s

 > Text split to sentences.
지금 이 음성은 저번에 사용한 티티에스 싸이트를 통해서 생성한 음성이 아니고,
샘플 음성을 제로샷으로 습득하여 로컬 티티에스 모델에서 만들어낸 목소리 입니다.
목소리를 잘 따라하는 것을 볼 수 있습니다.
 > ===========================


  0%|          | 0/3 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

캡슐화

In [1]:
import os
import shutil
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from melo.api import TTS

class Custom_TTS:
    def __init__(self, model_path='checkpoints_v2'):
        '''
        model_path: TTS를 위한 베이스 모델, 음성 변조를 위한 베이스 모델이 위치한 path
        '''
        print('다음 Repo.를 참조하여 개발한 모듈입니다: https://github.com/myshell-ai/OpenVoice')
        self.model_path = model_path

        # cuda 확인
        self.check_cuda()

    def check_cuda(self):
        '''cuda 환경 확인'''
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        print(f'사용 환경(cude): {self.device}')

    def set_model(self, language='KR'):
        '''
        모델 설정
        language: 언어 입력(en-au, en-br, en-default, en-india, en-newest, en-us, es, fr, jp, kr, zh)
        '''
        self.language = language
        
        # 톤 변경 모델 로드
        self.tone_color_converter = ToneColorConverter(f'{self.model_path}/converter/config.json', device=self.device)
        self.tone_color_converter.load_ckpt(f'{self.model_path}/converter/checkpoint.pth')
        print('톤 변경 모델 로드 완료')

        # TTS 모델 선언
        self.tts_model = TTS(language=self.language, device=self.device)
        print('TTS 모델 로드 완료')

        # 기본 화자 음성 임베딩: TTS 모델이 생성한 음성을 음색 변환 모델이 입력할 때, 원본 화자의 음색 정보를 제공함
        speaker_ids = self.tts_model.hps.data.spk2id
        for speaker_key in speaker_ids.keys():
            self.speaker_id = speaker_ids[speaker_key]
            speaker_key = speaker_key.lower().replace('_', '-')
        self.source_se = torch.load(f'{self.model_path}/base_speakers/ses/{speaker_key}.pth', map_location=self.device)
        print('기본 화자 음성 임베딩 완료')

    def get_reference_speaker(self, speaker_path, vad=True):
        '''
        흉내낼 목소리를 입력해주는 함수. 
        - 논문 상 최소 44초 길이 이상의 음성을 넣으라고 되어있음
        - base 목소리가 여자이기 때문에, 조금의 실험을 해본 결과 남자 목소리 보다는 여자 목소리를 더 잘 따라하는 경향을 보임
        - 꼭 mp3일 필요 없고 갤럭시 휴대폰 기본 녹음 포맷인 m4a도 문제 없었음

        path: 복사할 음성의 상대 경로를 입력
        vad: 목소리 감지 기능 켜기/끄기. 켤 경우 음성 내에서 목소리가 있는 부분만 전처리 함
        '''
        # 톤 컬러 임베딩
        self.target_se, audio_name = se_extractor.get_se(speaker_path, self.tone_color_converter, vad=vad)
        print('목소리 톤 임베딩 완료')

    def make_speech(self, text, output_path='output', speed=1.1):
        '''
        텍스트를 입력하면 TTS를 수행하는 함수. mp3를 생성하여 로컬에 저장함
        text: 변환을 원하는 언어를 입력
        output_path: TTS 결과물이 출력되는 경로
        speed: 음성 재생 속도. 1.1이 자연스러운 것 같음
        '''
        # 경로 설정, 기존 파일 존재시 삭제, 폴더 생성
        src_path = f'{output_path}/tmp.wav'
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.makedirs(output_path, exist_ok=True)

        # TTS 수행
        self.tts_model.tts_to_file(text, self.speaker_id, src_path, speed=speed)

        # 목소리 변조 수행
        self.tone_color_converter.convert(audio_src_path=src_path, 
                                          src_se=self.source_se, 
                                          tgt_se=self.target_se, 
                                          output_path=f'{output_path}/result.wav')

/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
tts_module = Custom_TTS()

다음 Repo.를 참조하여 개발한 모듈입니다: https://github.com/myshell-ai/OpenVoice
사용 환경(cude): cuda:0


In [3]:
tts_module.set_model()

/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
톤 변경 모델 로드 완료
TTS 모델 로드 완료
기본 화자 음성 임베딩 완료


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(ckpt_path, map_l

In [4]:
tts_module.get_reference_speaker(speaker_path='iena_sample.m4a')

OpenVoice version: v2


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[(0.0, 128.512)]
after vad: dur = 128.512


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/functional.py:666: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789115370/work/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


목소리 톤 임베딩 완료


In [5]:
tts_module.make_speech('오늘 회의시간에 너무 졸아서 고수석님께 혼났다')

 > Text split to sentences.
오늘 회의시간에 너무 졸아서 고수석님께 혼났다
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

check point 다운로드 테스트

In [10]:
import requests
from tqdm import tqdm
import zipfile
import os

url = "https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_v2_0417.zip"
filename = "checkpoints_v2_0417.zip"
extract_path = "checkpoints_v2"  # 압축을 풀 디렉토리

# HTTP 응답에서 Content-Length(파일 크기) 가져오기
response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))

# 다운로드 진행을 표시하는 tqdm 설정
with open(filename, "wb") as file, tqdm(
    desc=filename,
    total=total_size,
    unit='B',
    unit_scale=True,
    unit_divisor=1024,
) as bar:
    for data in response.iter_content(chunk_size=1024):
        file.write(data)
        bar.update(len(data))

print(f"{filename} 다운로드 완료!")

# 압축 해제 진행률 표시
with zipfile.ZipFile(filename, 'r') as zip_ref:
    # 압축된 파일의 전체 크기를 계산
    total_unzipped_size = sum((zinfo.file_size for zinfo in zip_ref.infolist()))

    # 압축 해제 진행률을 표시하는 tqdm 설정
    with tqdm(total=total_unzipped_size, unit='B', unit_scale=True, unit_divisor=1024, desc="Extracting") as bar:
        for zinfo in zip_ref.infolist():
            extracted_file_path = zip_ref.extract(zinfo, extract_path)
            # 압축 해제된 파일 크기만큼 진행률을 업데이트
            bar.update(zinfo.file_size)

print(f"{filename} 압축 해제 완료!")


checkpoints_v2_0417.zip: 100%|██████████| 116M/116M [00:13<00:00, 9.05MB/s] 


checkpoints_v2_0417.zip 다운로드 완료!


Extracting: 100%|██████████| 125M/125M [00:00<00:00, 168MB/s]

checkpoints_v2_0417.zip 압축 해제 완료!


다운로드 모듈 캡슐화

In [13]:
class Down_and_extract:
    def do(self, url, file_name, extract_path):
        try:

            # HTTP 응답에서 Content-Length(파일 크기) 가져오기
            response = requests.get(url, stream=True)
            total_size = int(response.headers.get('content-length', 0))

            # 다운로드 진행을 표시하는 tqdm 설정
            with open(filename, "wb") as file, tqdm(
                desc=filename,
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    file.write(data)
                    bar.update(len(data))

            print(f"{filename} 다운로드 완료!")

            # 압축 해제 진행률 표시
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                # 압축된 파일의 전체 크기를 계산
                total_unzipped_size = sum((zinfo.file_size for zinfo in zip_ref.infolist()))

                # 압축 해제 진행률을 표시하는 tqdm 설정
                with tqdm(total=total_unzipped_size, unit='B', unit_scale=True, unit_divisor=1024, desc="Extracting") as bar:
                    for zinfo in zip_ref.infolist():
                        extracted_file_path = zip_ref.extract(zinfo, extract_path)
                        # 압축 해제된 파일 크기만큼 진행률을 업데이트
                        bar.update(zinfo.file_size)
            print(f"{filename} 압축 해제 완료!")
            return True
        except:
            print('압축 해제 문제 발생')
            return False

url = "https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_v2_0417.zip"
filename = "checkpoints_v2_0417.zip"
extract_path = "checkpoints_v2"  # 압축을 풀 디렉토리

down_load = Down_and_extract()
down_load.do(url, filename, extract_path)

checkpoints_v2_0417.zip: 263B [00:00, 728kB/s]

checkpoints_v2_0417.zip 다운로드 완료!
압축 해제 문제 발생


False

# 전체 통합 모듈 테스트

In [10]:
import os
import shutil
import torch
import requests
from tqdm import tqdm
import zipfile
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from melo.api import TTS


class Custom_TTS:
    def __init__(self, model_path='checkpoints_v2'):
        '''
        model_path: TTS를 위한 베이스 모델, 음성 변조를 위한 베이스 모델이 위치한 path
        '''
        print('본 코드를 개발한 Repo. 입니다: https://github.com/Nyan-SouthKorea/RealTime_zeroshot_TTS_ko')
        print('다음 Repo.를 참조하여 개발한 모듈입니다: https://github.com/myshell-ai/OpenVoice')
        self.model_path = model_path

        # cuda 확인
        self.check_cuda()

    def check_cuda(self):
        '''cuda 환경 확인'''
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        print(f'사용 환경(cude): {self.device}')

    def checkpoint_download(self):
        '''
        모델의 pre-trained checkpoint가 있는지 확인하고 없으면 다운로드 함
        - 모델의 폴더만 확인하기 때문에, 폴더 안에 모델 변경이 있어도 유효성 검사를 수행하지 않음
        - 단순히 폴더가 없으면 다시 다운로드 하는 로직임
        '''
        if os.path.exists(self.model_path) == False:
            download = Down_and_extract()
            ret = download.do(url="https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_v2_0417.zip", filename="checkpoints_v2_0417.zip")
            if ret == False:
                with open('./error_txt.txt', 'r', encoding='utf-8-sig') as f:
                    error_txt = f.read()
                    print(error_txt)

    def set_model(self, language='KR'):
        '''
        모델 설정
        language: 언어 입력(en-au, en-br, en-default, en-india, en-newest, en-us, es, fr, jp, kr, zh)
        '''
        self.language = language
        
        # pre-trained 모델 다운로드
        self.checkpoint_download()

        # 톤 변경 모델 로드
        self.tone_color_converter = ToneColorConverter(f'{self.model_path}/converter/config.json', device=self.device)
        self.tone_color_converter.load_ckpt(f'{self.model_path}/converter/checkpoint.pth')
        print('톤 변경 모델 로드 완료')

        # TTS 모델 선언
        self.tts_model = TTS(language=self.language, device=self.device)
        print('TTS 모델 로드 완료')

        # 기본 화자 음성 임베딩: TTS 모델이 생성한 음성을 음색 변환 모델이 입력할 때, 원본 화자의 음색 정보를 제공함
        speaker_ids = self.tts_model.hps.data.spk2id
        for speaker_key in speaker_ids.keys():
            self.speaker_id = speaker_ids[speaker_key]
            speaker_key = speaker_key.lower().replace('_', '-')
        self.source_se = torch.load(f'{self.model_path}/base_speakers/ses/{speaker_key}.pth', map_location=self.device)
        print('기본 화자 음성 임베딩 완료')

    def get_reference_speaker(self, speaker_path, vad=True):
        '''
        흉내낼 목소리를 입력해주는 함수. 
        - 논문 상 최소 44초 길이 이상의 음성을 넣으라고 되어있음
        - base 목소리가 여자이기 때문에, 조금의 실험을 해본 결과 남자 목소리 보다는 여자 목소리를 더 잘 따라하는 경향을 보임
        - 꼭 mp3일 필요 없고 갤럭시 휴대폰 기본 녹음 포맷인 m4a도 문제 없었음

        path: 복사할 음성의 상대 경로를 입력
        vad: 목소리 감지 기능 켜기/끄기. 켤 경우 음성 내에서 목소리가 있는 부분만 전처리 함
        '''
        # 톤 컬러 임베딩
        self.target_se, audio_name = se_extractor.get_se(speaker_path, self.tone_color_converter, vad=vad)
        print('목소리 톤 임베딩 완료')

    def make_speech(self, text, output_path='output', speed=1.1):
        '''
        텍스트를 입력하면 TTS를 수행하는 함수. mp3를 생성하여 로컬에 저장함
        text: 변환을 원하는 언어를 입력
        output_path: TTS 결과물이 출력되는 경로
        speed: 음성 재생 속도. 1.1이 자연스러운 것 같음
        '''
        # 경로 설정, 기존 파일 존재시 삭제, 폴더 생성
        src_path = f'{output_path}/tmp.wav'
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.makedirs(output_path, exist_ok=True)

        # TTS 수행
        self.tts_model.tts_to_file(text, self.speaker_id, src_path, speed=speed)

        # 목소리 변조 수행
        self.tone_color_converter.convert(audio_src_path=src_path, 
                                          src_se=self.source_se, 
                                          tgt_se=self.target_se, 
                                          output_path=f'{output_path}/result.wav')

class Down_and_extract:
    def do(self, url, filename):
        try:
            # HTTP 응답에서 Content-Length(파일 크기) 가져오기
            response = requests.get(url, stream=True)
            total_size = int(response.headers.get('content-length', 0))

            # 다운로드 진행을 표시하는 tqdm 설정
            with open(filename, "wb") as file, tqdm(
                desc=filename,
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    file.write(data)
                    bar.update(len(data))

            print(f"{filename} 다운로드 완료!")

            # 압축 해제 진행률 표시
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                # 압축된 파일의 전체 크기를 계산
                total_unzipped_size = sum((zinfo.file_size for zinfo in zip_ref.infolist()))

                # 압축 해제 진행률을 표시하는 tqdm 설정
                with tqdm(total=total_unzipped_size, unit='B', unit_scale=True, unit_divisor=1024, desc="Extracting") as bar:
                    for zinfo in zip_ref.infolist():
                        extracted_file_path = zip_ref.extract(zinfo, './')
                        # 압축 해제된 파일 크기만큼 진행률을 업데이트
                        bar.update(zinfo.file_size)
            print(f"{filename} 압축 해제 완료!")
            return True
        except Exception as e:
            print(f'압축 해제 문제 발생: \n{e}')
            return False

In [11]:
tts_module = Custom_TTS()

본 코드를 개발한 Repo. 입니다: https://github.com/Nyan-SouthKorea/RealTime_zeroshot_TTS_ko
다음 Repo.를 참조하여 개발한 모듈입니다: https://github.com/myshell-ai/OpenVoice
사용 환경(cude): cuda:0


In [12]:
tts_module.set_model()

checkpoints_v2_0417.zip: 100%|██████████| 116M/116M [00:14<00:00, 8.15MB/s] 


checkpoints_v2_0417.zip 다운로드 완료!


Extracting: 100%|██████████| 125M/125M [00:00<00:00, 180MB/s]
/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


checkpoints_v2_0417.zip 압축 해제 완료!


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(resume_path, 

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
톤 변경 모델 로드 완료
TTS 모델 로드 완료
기본 화자 음성 임베딩 완료


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(ckpt_path, map_l

In [13]:
tts_module.get_reference_speaker(speaker_path='iena_sample.m4a')

OpenVoice version: v2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/ubuntu/anaconda3/envs/openvoice/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[(0.0, 128.512)]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
after vad: 

In [14]:
tts_module.make_speech('오늘 회의시간에 너무 졸아서 고수석님께 혼났다')

 > Text split to sentences.
오늘 회의시간에 너무 졸아서 고수석님께 혼났다
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  3.43it/s]
